In [ ]:
import re
import sys
import csv
import numpy as np
import pandas as pd

In [ ]:
# imports required for model
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
from keras import backend as K

from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

!pip install seqeval
from seqeval.metrics import precision_score,recall_score,f1_score,classification_report

Instructions for updating:
non-resource variables are not supported in the long term
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=cd390175ff73add2837b560996eb8dda1ca988e6ee0afd026389e2934e689b61
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from google.colab import files
uploaded = files.upload()

# io.BytesIO(uploaded['emails.csv']) 
# emails.rar is compressed emails.csv Enron Email Dataset
!unrar e emails.rar 

Saving emails.rar to emails.rar


In [ ]:
csv.field_size_limit(sys.maxsize)

dataset = pd.read_csv('emails.csv',engine='python',encoding='utf-8',error_bad_lines=False)

print(dataset.shape)
dataset.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
emails = dataset['message']

print(emails.shape)
emails.head()

(517401,)


0    Message-ID: <18782981.1075855378110.JavaMail.e...
1    Message-ID: <15464986.1075855378456.JavaMail.e...
2    Message-ID: <24216240.1075855687451.JavaMail.e...
3    Message-ID: <13505866.1075863688222.JavaMail.e...
4    Message-ID: <30922949.1075863688243.JavaMail.e...
Name: message, dtype: object

In [ ]:
print(emails[0])
print(type(emails[0]))

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 
<class 'str'>


In [ ]:
def tokenizer(sentence):
  result = []
  sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',sentence)
  if len(sentence)>1 :
    for sen in sentences:
      sen = re.sub(r'\s+',' ',sen)       # replacing extra spaces with single space
      sen = sen.strip()
      if sen.endswith('.'):
        sen = sen[:-1]
      result.append(sen)    
  return result      


In [ ]:
def wordIndex(words,index,count):
  for word in words:
    s = word.lower()
    if s not in index.keys():
      count+=1
      index[s] = count
    
  return count 

In [ ]:
def encodeSequence(sentence,word_index):
  sequence = []
  for word in sentence:
    s = word
    if s in word_index.keys():
      sequence.append(word_index[s])  
  return sequence 

In [ ]:
def padSequence(sequence,max_length,word):
  if max_length <= len(sequence):
    return sequence[0:max_length]   
  seq = sequence.copy()
  if max_length>len(sequence) :
    for i in range(0,max_length-len(sequence)):
      seq.append(word)      
  return seq    

In [ ]:
from google.colab import files
uploaded = files.upload()

!unzip dataset.zip

Saving dataset.zip to dataset.zip
Archive:  dataset.zip
  inflating: dataset.json            


In [ ]:
import json

f = open("dataset.json",'r',encoding='utf-8')
Dataset = json.load(f)
f.close()

print(Dataset[0])

[['Quarterly', 'B DATE'], ['profits', 'O '], ['at', 'O '], ['US', 'B GPE'], ['media', 'O '], ['giant', 'O '], ['TimeWarner', 'B ORG'], ['jumped', 'O '], ['76', 'B PERCENT'], ['%', 'I PERCENT'], ['to', 'O '], ['$', 'O '], ['1.13bn', 'B MONEY'], ['(', 'O '], ['£', 'O '], ['600', 'B MONEY'], ['m', 'O '], [')', 'O '], ['for', 'O '], ['the', 'B DATE'], ['three', 'I DATE'], ['months', 'I DATE'], ['to', 'I DATE'], ['December', 'I DATE'], [',', 'O '], ['from', 'O '], ['$', 'O '], ['639', 'B MONEY'], ['m', 'O '], ['year', 'B DATE'], ['-', 'I DATE'], ['earlier', 'I DATE']]


In [ ]:
x_data = []
y_data = []

for i in range(len(Dataset)):
  x = []
  y = []
  for j in range(len(Dataset[i])): 
    x.append(Dataset[i][j][0])
    y.append(Dataset[i][j][1])
  x_data.append(x)
  y_data.append(y)

print(x_data[0])
print(y_data[0])

['Quarterly', 'profits', 'at', 'US', 'media', 'giant', 'TimeWarner', 'jumped', '76', '%', 'to', '$', '1.13bn', '(', '£', '600', 'm', ')', 'for', 'the', 'three', 'months', 'to', 'December', ',', 'from', '$', '639', 'm', 'year', '-', 'earlier']
['B DATE', 'O ', 'O ', 'B GPE', 'O ', 'O ', 'B ORG', 'O ', 'B PERCENT', 'I PERCENT', 'O ', 'O ', 'B MONEY', 'O ', 'O ', 'B MONEY', 'O ', 'O ', 'O ', 'B DATE', 'I DATE', 'I DATE', 'I DATE', 'I DATE', 'O ', 'O ', 'O ', 'B MONEY', 'O ', 'B DATE', 'I DATE', 'I DATE']


In [ ]:
max_len = 50
encoded_x_data = []

for data in x_data:
  encoded_x_data.append(padSequence(data,max_len,'PADword'))

print(len(encoded_x_data[3]))
print(x_data[3])
print(encoded_x_data[3])

50
['Its', 'profits', 'were', 'buoyed', 'by', 'one', '-', 'off', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'Warner', 'Bros', ',', 'and', 'less', 'users', 'for', 'AOL']
['Its', 'profits', 'were', 'buoyed', 'by', 'one', '-', 'off', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'Warner', 'Bros', ',', 'and', 'less', 'users', 'for', 'AOL', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword']


In [ ]:
tags={}
i=0
for data in y_data:
  for word in data:
    if word not in tags.keys():
      tags[word] = i
      i+=1

print(len(tags.keys()))      
tags

36


{'B CARDINAL': 8,
 'B DATE': 0,
 'B EVENT': 19,
 'B FAC': 26,
 'B GPE': 2,
 'B LANGUAGE': 34,
 'B LAW': 28,
 'B LOC': 20,
 'B MONEY': 6,
 'B NORP': 14,
 'B ORDINAL': 12,
 'B ORG': 3,
 'B PERCENT': 4,
 'B PERSON': 11,
 'B PRODUCT': 17,
 'B QUANTITY': 21,
 'B TIME': 23,
 'B WORK_OF_ART': 31,
 'I CARDINAL': 16,
 'I DATE': 7,
 'I EVENT': 27,
 'I FAC': 33,
 'I GPE': 15,
 'I LAW': 29,
 'I LOC': 25,
 'I MONEY': 10,
 'I NORP': 30,
 'I ORDINAL': 35,
 'I ORG': 9,
 'I PERCENT': 5,
 'I PERSON': 13,
 'I PRODUCT': 18,
 'I QUANTITY': 22,
 'I TIME': 24,
 'I WORK_OF_ART': 32,
 'O ': 1}

In [ ]:
i = 0
for x in y_data:
  seq = padSequence(encodeSequence(x,tags),max_len,tags['O '])
  if i==0:
    encoded_y_data = np.array(seq,dtype='int32')
    i+=1
  else:  
    encoded_y_data = np.vstack((encoded_y_data,seq))

print(len(encoded_y_data[3]))
print(y_data[3])
print(encoded_y_data[3])

50
['O ', 'O ', 'O ', 'O ', 'O ', 'B CARDINAL', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B ORG', 'I ORG', 'O ', 'O ', 'O ', 'O ', 'O ', 'B ORG']
[1 1 1 1 1 8 1 1 1 1 1 1 1 1 1 3 9 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(encoded_x_data, encoded_y_data, test_size=0.1, random_state=0)
sess = tf.Session()
K.set_session(sess)
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
batch_size = 25
def ElmoEmbedding(x):
    return elmo_model(inputs={"tokens": tf.squeeze(tf.cast(x,tf.string)),"sequence_len": tf.constant(batch_size*[max_len])},
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
n_tags = len(tags.keys())
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len,1024))(input_text)

x = Bidirectional(LSTM(units=512, return_sequences=True,recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,recurrent_dropout=0.2, dropout=0.2))(x)

x = add([x, x_rnn])  
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
x_train, x_validate = x_train[:1213*batch_size], x_train[-135*batch_size:]
y_train, y_validate = y_train[:1213*batch_size], y_train[-135*batch_size:]
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)
y_validate = y_validate.reshape(y_validate.shape[0], y_validate.shape[1], 1)
history = model.fit(np.array(x_train), y_train, validation_data=(np.array(x_validate), y_validate),batch_size=batch_size, epochs=3, verbose=1)

Train on 30325 samples, validate on 3375 samples
Epoch 1/3
30325/30325 [==============================] - ETA: 0s - loss: 0.0889 - acc: 0.9741

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


30325/30325 [==============================] - 1422s 47ms/sample - loss: 0.0889 - acc: 0.9741 - val_loss: 0.0612 - val_acc: 0.9805
Epoch 2/3
30325/30325 [==============================] - 1400s 46ms/sample - loss: 0.0528 - acc: 0.9829 - val_loss: 0.0558 - val_acc: 0.9818
Epoch 3/3
30325/30325 [==============================] - 1400s 46ms/sample - loss: 0.0410 - acc: 0.9862 - val_loss: 0.0542 - val_acc: 0.9826


In [ ]:
def testToLabel(index,list1):
  keys = list(index.keys())
  values = list(index.values())
  ans = []
  for i in range(len(list1)):
    sub = []
    for j in range(len(list1[i])):
      pos = values.index(list1[i][j])
      if "O " != keys[pos] :
        val = keys[pos].split(" ")[0] + keys[pos].split(" ")[1]
      else :
        val = "PADword"
      sub.append(val)
    ans.append(sub)    
  return ans

def predictToLabel(index,list1):
  keys = list(index.keys())
  values = list(index.values())
  ans = []
  for i in range(len(list1)):
    sub = []
    for j in range(len(list1[i])):
      p = np.argmax(list1[i][j])
      pos = values.index(p)
      if "O " != keys[pos] :   
        val = keys[pos].split(" ")[0] + keys[pos].split(" ")[1]
      else :
        val = "PADword"
      sub.append(val)
    ans.append(sub)    
  return ans

In [ ]:
x_test = x_test[:149*batch_size]
prediction = model.predict(np.array(x_test),batch_size=batch_size ,verbose=1)

NameError: ignored

In [ ]:
pred_labels = predictToLabel(tags,prediction)
test_labels = testToLabel(tags,y_test[:149*batch_size])
print(classification_report(test_labels,pred_labels))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OO seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BDATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: IDATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BORDINAL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BGPE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: 

              precision    recall  f1-score   support

    CARDINAL       0.80      0.91      0.85       741
        DATE       0.88      0.89      0.89      1157
       EVENT       0.40      0.69      0.51        52
         FAC       0.33      0.36      0.34        59
         GPE       0.85      0.88      0.86      1062
    LANGUAGE       0.67      1.00      0.80         4
         LAW       0.25      0.31      0.28        16
         LOC       0.82      0.76      0.79       102
       MONEY       0.73      0.78      0.75       257
        NORP       0.94      0.84      0.89       394
     ORDINAL       0.96      0.93      0.95       227
         ORG       0.67      0.74      0.70      1619
     PERCENT       0.97      0.97      0.97       178
      PERSON       0.73      0.83      0.78      1698
     PRODUCT       0.31      0.28      0.30        60
    QUANTITY       0.19      0.43      0.27        30
        TIME       0.68      0.85      0.75        67
 WORK_OF_ART       0.19    

In [ ]:
def extractHeader(string,header):
  s = string.split(header)[1]
  if s.find('<')!=-1:
    s = s.split('<')[0]
  return s.strip()

In [ ]:
def extractBody(string,lastheaderline):
  i = string.find(lastheaderline)
  return string[i+len(lastheaderline):].strip()

In [ ]:
def featureExtraction(Email):
  
  L = Email.splitlines(True)
  X_From = extractHeader(L[8],'X-From:') 
  X_To = extractHeader(L[9],'X-To:')
  X_Origin = extractHeader(L[13],'X-Origin:')
  X_cc= extractHeader(L[10],'X-cc:')
  X_bcc= extractHeader(L[11],'X-bcc:') 
  subject = extractHeader(L[4],'Subject:') 
  message = extractBody(Email,L[14])  # first 14 lines contains headers 

  result = []
  string = X_From+" "+X_To+" "+X_Origin+" "+X_cc+" "+X_bcc+" "+subject 
  for token in tokenizer(string):
    result.append(token)
  for token in tokenizer(message):
    result.append(token)

  return result

feature = featureExtraction(emails[0])

for i in range(len(feature)):
  feature[i]= feature[i].split(" ")

print(feature)

[['Phillip', 'K', 'Allen', 'Tim', 'Belden', 'Allen-P'], ['Here', 'is', 'our', 'forecast']]


In [ ]:
encoded_feature = []

for data in feature:
  encoded_feature.append(padSequence(data,max_len,'PADword'))

print(encoded_feature)

[['Phillip', 'K', 'Allen', 'Tim', 'Belden', 'Allen-P', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword'], ['Here', 'is', 'our', 'forecast', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword

In [ ]:
test = encoded_feature.copy()
for i in range(0,batch_size-len(encoded_feature)) :
  test.append(['PADword']*max_len)

In [ ]:
prediction = model.predict(np.array(test),batch_size=batch_size,verbose=1)

In [ ]:
def predictToIOBLabel(index,list1):
  keys = list(index.keys())
  values = list(index.values())
  ans = []
  for i in range(len(list1)):
    sub = []
    for j in range(len(list1[i])):
      p = np.argmax(list1[i][j])
      pos = values.index(p)
      sub.append(keys[pos])
    ans.append(sub)    
  return ans
  
prediction = predictToIOBLabel(tags,prediction[:len(encoded_feature)])

In [ ]:
for i in range(len(feature)):
  for j in range(len(feature[i])):
    print(feature[i][j],'-',prediction[i][j])
  print()    

Phillip - B PERSON
K - I PERSON
Allen - I PERSON
Tim - I PERSON
Belden - I PERSON
Allen-P - I PERSON

Here - O 
is - O 
our - O 
forecast - O 

